###We are going to learn some advanced options available in the different file/table read functions (Least bother)

#### Sample data
#####This is a retail data
productId,productName,stdCost,stdPrice,effDt,dt
PURA100,"Pure Soft Detergent - ~"100ml~"", 1.50 , 3.00,2019-12-03 00:00:01,2019-12-01 <br>
PURA200,"Pure Soft ","Detergent","- 200ml", 2.00 , 3.99,2019-03-01 00:00:01,2019-01-01  <br>
PURA250,"Pure Soft Detergent - 250ml", 2.30 , 4.50,2019-03-01 00:00:01,2019-01-01 <br>
PURA500,"Pure Soft Detergent - 500ml", 3.50 , 6.50,2019-03-02 00:00:01,2019-01-01 <br>
DETA100,"Detafast Stain Remover - 100ml", 3.00 , 6.00,2019-03-01 00:00:01,2019-01-01 <br>
DETA200,"Detafast Stain Remover - 200ml", 3.50 , 6.50,2019-03-03 00:00:01,2019-01-01 <br>
DETA800,na, 6.00 ,-1,2019-03-04 00:00:01,2019-01-01 <br>
SUPA101,"Super Soft - Product Sample", 0.30 ,0,2019-03-06 00:00:01,2019-01-01 <br>
SUPA102,"Super Soft - 250ml ", 2.50 , 4.50,2019-03-01 00:00:01,2019-01-01 <br>
SUPA103,, 3.50 , 6.99 ,2019-04-01 00:00:01,2019-01-01
,"Super Soft - 1 Litre", 5.00 , 9.99,2019-04-02 00:00:01,2019-04-20,2019-01-01 <br>
SUPA105 , "Super Soft Bulk - 2 Litres"   , 8.00 , 14.50,2019-04-03 00:00:01,2019-01-01  <br>
Deta10,"Pure Soft Detergent - ~"100ml~", 1.50 ,3.00,2019-12-03 00:00:01,2019-12-01  <br>
#end of data

#### Important - There are 3 typical read modes and the default read mode is permissive.
##### 1. permissive — All fields are set to null and corrupted records are placed in a string column called _corrupt_record
##### 2. dropMalformed — Drops all rows containing corrupt records.
##### 3. failFast — Fails when corrupt records are encountered.

####CSV Advanced Feature - Very Important, Important, Not Important (just try to know once for [all](url))
**Very Important:** path, schema, sep, header, inferSchema, samplingRatio, mode, columnNameOfCorruptRecord <br>
**Important:** quote, escape,comment, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, dateFormat, timestampFormat, multiLine, enforceSchema,pathGlobFilter, recursiveFileLookup,modifiedBefore,modifiedAfter<br>
**Not Important:** encoding, positiveInf, negativeInf, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition, charToEscapeQuoteEscaping, emptyValue, locale, lineSep, unescapedQuoteHandling

In [0]:
struct1="cid string,name string,amt string,dop string"
df1=spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/malformeddata.txt",header=False,sep=",",comment='#') #permit malformed records and allow me to do a RCA (Root cause analysis)
df1.where("upper(_c2)=lower(_c2)").show()

In [0]:
#**Very Important:** path, schema, sep, header, inferSchema, samplingRatio, mode, columnNameOfCorruptRecord <br>
struct1="cid int,name string,amt decimal(10,2),dop date"
df1=spark.read.schema(struct1).csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/malformeddata.txt",header=False,sep="," ,comment='#',mode='dropMalformed')#Blindly drop malformed records\n",
display(df1)                                   
struct1="cid int,name string,amt decimal(10,2),dop date,corrupt_record string"
df1=spark.read.schema(struct1).csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/malformeddata.txt",header=False,sep=",",comment='#',mode='permissive',columnNameOfCorruptRecord='corrupt_record')#permit malformed records and allow me to do a RCA (Root cause analysis)
display(df1.where("corrupt_record is not null"))
df1=spark.read.schema(struct1).csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/malformeddata.txt",header=False,sep=",",comment='#',mode='failFast')#Fail immediately when malformed records occurs
display(df1)

In [0]:
#**Important:** quote, escape,comment, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace,  dateFormat, timestampFormat, multiLine, enforceSchema,pathGlobFilter, recursiveFileLookup,modifiedBefore,modifiedAfter, nullValue, nanValue<br>\n",
struct1="cid int,name string,amt decimal(10,2),dop date"
df1=spark.read.schema(struct1).csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/malformeddata1.txt",header=False,sep=",",comment='#',quote="'",escape='|',ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True,dateFormat='yyyy-dd-MM',mode="permissive",modifiedAfter='2025-12-20',multiLine=True,nullValue='na')
display(df1)

####JSON Advanced Feature - <br>
  **Very Important** - path,schema,columnNameOfCorruptRecord,dateFormat,timestampFormat,multiLine,pathGlobFilter,recursiveFileLookup<br>
  No header, No inferSchema, No sep in json...<br>
  **Important** - primitivesAsString(consider all cols as string), prefersDecimal, allowComments, allowUnquotedFieldNames, allowSingleQuotes, lineSep, samplingRatio, dropFieldIfAllNull, modifiedBefore, modifiedAfter, useUnsafeRow(This is performance optimization when the data is loaded into spark memory) <br>
  **Not Important** (just try to know once for all) - allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, allowUnquotedControlChars, encoding, locale, allowNonNumericNumbers<br>
   

In [0]:
df_json=spark.read.json("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/simple_json.txt",samplingRatio=0.1,prefersDecimal=True)
#primitivesAsString
df_json.printSchema()
df_json.show()

In [0]:
strt1="id int,name string,amt float, dop date,custom_corruptrow string"
df_json=spark.read.json("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/simple_json2.txt",allowComments=True,lineSep='~',allowSingleQuotes=True,allowUnquotedFieldNames=True,columnNameOfCorruptRecord="custom_corruptrow",schema=strt1,dateFormat='yyyy-dd-MM')
#primitivesAsString
df_json.printSchema()
df_json.show(10,False)

In [0]:
strt1="id int,name string,amt float, dop date,custom_corruptrow string"
df_json=spark.read.json("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/simple_json_multiline3.txt",allowComments=True,allowSingleQuotes=True,allowUnquotedFieldNames=True,columnNameOfCorruptRecord="custom_corruptrow",schema=strt1,dateFormat='yyyy-dd-MM',multiLine=True)
#primitivesAsString
df_json.printSchema()
df_json.show(10,False)

####Serialized data Advanced Feature - orc, parquet/delta (very very important & we learn indepth)
- PathOrPaths
- **mergeSchema** - Important interview property (make it proactive/make it driven in the interview) SCHEMA EVOLUTION
- pathGlobFilter
- recursiveFileLookup
- modifiedBefore
- modifiedAfter

Problem statement:<br>
Source is sending data in any way they want...<br>
Day1/source1- 5 cols <br>
Day2/source2 - 7 Cols

1. I am reading the dataframe in csv/json...
2. Writing into a orc/parquet format in a single location.
3. Reading data in a orc/parquet format using mergeSchema option.

In [0]:
df1=spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/",inferSchema=True,header=True)
display(df1)

problem:
Source is sending data in any way they want...
Day1/source1- 5 cols
Day2/source2 - 7 Cols\n",
1. I am reading the dataframe in csv/json...
2. Writing into a orc/parquet format in a single location.
3. Reading data in a orc/parquet format using mergeSchema option.

In [0]:
#I am converting the CSV data into ORC data format to achieve SCHEMA EVOLUTION(CHANGING)
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source1.txt",inferSchema=True,header=True).write.orc("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/orc_targetdata_merged/",mode='append')
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source2.txt",inferSchema=True,header=True).write.orc("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/orc_targetdata_merged/",mode='append')
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source3.txt",inferSchema=True,header=True).write.orc("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/orc_targetdata_merged/",mode='append')

In [0]:
orc_merged_data=spark.read.orc("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/orc_targetdata_merged/",mergeSchema=True)
display(orc_merged_data)

In [0]:
#I am converting the CSV data into Parquet data format to achieve SCHEMA EVOLUTION(CHANGING)
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source1.txt",inferSchema=True,header=True).write.parquet("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/parquet_targetdata_merged/")
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source2.txt",inferSchema=True,header=True).write.parquet("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/parquet_targetdata_merged/",mode='append')
spark.read.csv("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/sampledata/source3.txt",inferSchema=True,header=True).write.parquet("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/parquet_targetdata_merged/",mode='append')

In [0]:
df_parquet=spark.read.parquet("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/serialized_compressed_data_sources/parquet_targetdata_merged/",mergeSchema=True)
display(df_parquet)